In [1]:
from tqdm import tqdm
import torch
import h5py
import torch.nn.functional as F

In [2]:
img = {}
motion = {}
with h5py.File('../data/MSVD/features/MSVD_visual_clip.hdf5', 'r') as fs:
    for key in fs.keys():
        img[key] = fs[key][()]

with h5py.File('../data/MSVD/features/MSVD_motion_clip.hdf5', 'r') as fs:
    for key in fs.keys():
        motion[key] = fs[key][()]

In [3]:
max_frame = 20

In [4]:
with h5py.File("../data/MSVD/features/MSVD_edge_clip.hdf5", "w") as fs:
    for video_id in tqdm(img.keys()):

        idx = torch.arange(max_frame)
        edge_index = torch.cat([
            torch.stack([idx, idx], dim=0),   # (2, 20)
            torch.stack([idx[:-1], idx[1:]], dim=0)    # (2, 19)
        ], dim=1)

        # =====> MOTION <=====
        motion_feature = torch.from_numpy(motion[video_id])
        num_frames, feat_dim = motion_feature.shape
        pad_len = max_frame - num_frames
        
        if pad_len < 0:
            motion_feature = motion_feature[:max_frame]
        else:
            pad_feat = torch.zeros((pad_len, feat_dim), dtype=motion_feature.dtype)
            motion_feature = torch.cat([motion_feature, pad_feat], dim=0)


        # =====> VISUAL <=====
        visual_feature = torch.from_numpy(img[video_id])
        norm_a = torch.norm(visual_feature[:-1], dim=1, keepdim=True)
        norm_b = torch.norm(visual_feature[1:], dim=1, keepdim=True)
        upper_diag = (visual_feature[:-1] * visual_feature[1:]) / ((norm_a * norm_b))

        num_frames, feat_dim = upper_diag.shape
        pad_len = max_frame - num_frames - 1
        if pad_len < 0:
            upper_diag = upper_diag[:max_frame, :]
        else:
            pad_feat = torch.zeros((pad_len, feat_dim), dtype=upper_diag.dtype)
            upper_diag = torch.cat([upper_diag, pad_feat], dim=0)
        pad = torch.zeros(upper_diag.shape[0], motion_feature.shape[1] - upper_diag.shape[1])
        upper_diag = torch.cat([upper_diag, pad], dim=1)
        
        
        # Edge Graph
        edge_attr = torch.zeros(edge_index.shape[1], motion_feature.shape[1])
        edge_attr[:20] = motion_feature
        edge_attr[20:] = upper_diag
        grp = fs.create_group(video_id)
        grp.create_dataset("edge_index", data=edge_index.numpy())
        grp.create_dataset("edge_attr", data=edge_attr.numpy())

100%|██████████| 1970/1970 [00:15<00:00, 130.93it/s]
